In [240]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
vData = pd.read_csv("voting_data.csv")
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

In [241]:
vData

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,?,?,?,?,n,y,y,y,n,y


In [242]:
#Cleaning data

vData = vData.replace(to_replace = 'y', value = 1)
vData = vData.replace(to_replace = 'n', value = 0)
vData = vData.replace(to_replace = 'republican', value = 1)
vData = vData.replace(to_replace = 'democrat', value = 0)
vData.rename(columns={'Class Name':'Class'}, inplace=True)
vData

,Class,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,1,0,1,0,1,1,1,0,0,0,1,?,1,1,1,0,1
1,1,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,?
2,0,?,1,1,?,1,1,0,0,0,0,1,0,1,1,0,0
3,0,0,1,1,0,?,1,0,0,0,0,1,0,1,0,0,1
4,0,1,1,1,0,1,1,0,0,0,0,1,?,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,1,0,0,1,1,1,1,0,0,1,1,0,1,1,1,0,1
431,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,0,1
432,1,0,?,0,1,1,1,0,0,0,0,1,1,1,1,0,1
433,1,0,0,0,1,1,1,?,?,?,?,0,1,1,1,0,1


In [243]:
def compare_dt_nb(data, missing):
    
    #replacing missing values
    vData = data
#Replaces missing values with a 2
    if missing == 1:
        vData = vData.replace(to_replace = '?', value = 2)
        results = "Ternary Results"
    
#Drops all missing values from the data
    elif missing == 2:
        vData = vData.replace(to_replace = '?', value = pd.NaT)
        vData = vData.dropna()
        results = "Dropping missing values results"

#Replaces all missing values with it's most common value    
    else:
        for c in vData.columns:
            if c == 'Class':
                continue
            else:
                current = vData[c].value_counts()
                if current[0] < current[1]:
                    vData[c] = vData[c].replace('?', 1)
                else:
                    vData[c] = vData[c].replace('?', 0)
        results = "Replacing missing values with most common value"
        
    x = vData.drop('Class', axis=1)
    y = vData['Class']

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=1)
    clf = DecisionTreeClassifier()

    clf = clf.fit(x_train, y_train)
    y_predictions_dt = clf.predict(x_test)
    
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    y_predictions_nb = gnb.predict(x_test)
    
    dt_scores = cross_val_score(clf, x, y, cv=5, scoring = 'f1_micro')
    print(results)
    print(metrics.classification_report(y_test, y_predictions_dt))
    print(metrics.classification_report(y_test, y_predictions_nb))
    
    print("")
    print("Cross val scores")
    print("dt scores")
    dt_scores = cross_val_score(clf, x, y, cv=5, scoring = 'f1_micro')
    print(dt_scores)
    mean_dt_score = round(dt_scores.mean(), 3)
    print(mean_dt_score)
    
    print("")
    print("nb scores")
    nb_scores = cross_val_score(gnb, x, y, cv=5, scoring='f1_micro')
    print(nb_scores)
    mean_nb_score = round(nb_scores.mean(), 3)
    print(mean_nb_score)


In [244]:

compare_dt_nb(vData, 1)


Ternary Results
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        56
           1       0.94      0.94      0.94        31

    accuracy                           0.95        87
   macro avg       0.95      0.95      0.95        87
weighted avg       0.95      0.95      0.95        87

              precision    recall  f1-score   support

           0       0.98      0.95      0.96        56
           1       0.91      0.97      0.94        31

    accuracy                           0.95        87
   macro avg       0.95      0.96      0.95        87
weighted avg       0.96      0.95      0.95        87


Cross val scores
dt scores
[0.97701149 0.95402299 0.96551724 0.91954023 0.94252874]
0.952

nb scores
[0.93103448 0.95402299 0.93103448 0.97701149 0.89655172]
0.938


In [245]:
compare_dt_nb(vData, 2)

Dropping missing values results
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        23
           1       1.00      0.92      0.96        24

    accuracy                           0.96        47
   macro avg       0.96      0.96      0.96        47
weighted avg       0.96      0.96      0.96        47

              precision    recall  f1-score   support

           0       0.85      1.00      0.92        23
           1       1.00      0.83      0.91        24

    accuracy                           0.91        47
   macro avg       0.93      0.92      0.91        47
weighted avg       0.93      0.91      0.91        47


Cross val scores
dt scores
[0.95744681 0.95744681 0.97826087 0.95652174 0.89130435]
0.948

nb scores
[0.93617021 0.93617021 0.97826087 1.         0.91304348]
0.953


In [246]:
compare_dt_nb(vData, 3)

Replacing missing values with most common value
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        56
           1       0.94      0.94      0.94        31

    accuracy                           0.95        87
   macro avg       0.95      0.95      0.95        87
weighted avg       0.95      0.95      0.95        87

              precision    recall  f1-score   support

           0       0.98      0.91      0.94        56
           1       0.86      0.97      0.91        31

    accuracy                           0.93        87
   macro avg       0.92      0.94      0.93        87
weighted avg       0.94      0.93      0.93        87


Cross val scores
dt scores
[0.96551724 0.94252874 0.95402299 0.94252874 0.93103448]
0.947

nb scores
[0.94252874 0.89655172 0.94252874 0.95402299 0.86206897]
0.92


In [247]:
#After comparing my results, the naïve bayes classification appears to be inferior to the decision tree classification
# due to the low f-1 scores for each way I handled the missing values. Overall, there was very little change in the decision 
# tree algorithm on how I handled the missing values. However, based the f1-scores, I get the best results from dropping the 
# missing values for the decision tree algorigthm. I didn't expect that when I first started the assignment. For
# the naïve bayes classification, the best results were provided by handling the missing values as a ternary value. 